इस लेख में शामिल जप्टर नोटबुक [अध्याय 4 कोड आधार](https://github.com/datawhalechina/learn-nlp-with-transformers/tree/main/docs/%E7%AF%87%E7%AB) में है %A04 -%E4%BD%BF%E7%94%A8ट्रांसफॉर्मर%E8%A7%A3%E5%86%B3NLP%E4%BB%BB%E5%8A%A1)।

इस ट्यूटोरियल को सीधे खोलने के लिए Google Colab नोटबुक का उपयोग करने की अनुशंसा की जाती है, ताकि आप प्रासंगिक डेटा सेट और मॉडल जल्दी से डाउनलोड कर सकें।
यदि आप इस नोटबुक को Google के कोलाब में खोल रहे हैं, तो आपको ट्रांसफॉर्मर और 🤗डेटासेट लाइब्रेरी इंस्टॉल करने की आवश्यकता हो सकती है। इसे स्थापित करने के लिए निम्नलिखित कमांड को अनकम्मेंट करें।

In [1]:
# !पिप डेटासेट ट्रांसफार्मर स्थापित करें

# मशीन प्रश्न उत्तर कार्यों पर ट्रांसफार्मर मॉडल को फाइन-ट्यून करें

इस नोटबुक में, हम सीखेंगे कि मशीन प्रश्न उत्तर कार्यों को हल करने के लिए [🤗 ट्रांसफॉर्मर्स] (https://github.com/huggingface/transformers) के ट्रांसफॉर्मर मॉडल को कैसे ठीक किया जाए। यह लेख मुख्य रूप से निष्कर्षण प्रश्न और उत्तर कार्य को हल करता है: एक प्रश्न और पाठ का एक टुकड़ा दिया गया है, उस पाठ खंड (स्पैन) को ढूंढें जो पाठ से प्रश्न का उत्तर दे सकता है। `ट्रेनर` एपीआई और डेटासेट पैकेज का उपयोग करके, हम आसानी से डेटासेट लोड करेंगे और फिर ट्रांसफार्मर को फाइन-ट्यून करेंगे। नीचे दिया गया चित्र एक सरल उदाहरण देता है
![QA कार्य का प्रतिनिधित्व करने वाला विजेट अनुमान](images/question_answering.png)

**नोट:** नोट: इस लेख का प्रश्न और उत्तर कार्य पाठ से उत्तर निकालना है, न कि सीधे उत्तर उत्पन्न करना!

इस नोटबुक में डिज़ाइन किए गए उदाहरणों का उपयोग SQUAD 1 और SQUAD 2 के समान किसी भी निष्कर्षण प्रश्न का उत्तर देने वाले कार्यों को हल करने के लिए किया जा सकता है, और [मॉडल हब] (https://huggingface.co/models) में किसी भी मॉडल चेकपॉइंट का उपयोग कर सकते हैं, जब तक इन मॉडल में एक टोकन वर्गीकरण प्रमुख और एक तेज़ टोकननाइज़र शामिल है। मॉडल और तेज़ टोकननाइज़र के बीच संबंधित संबंध के लिए, देखें: [यह तालिका](https://huggingface.co/transformers/index.html#bigtable)।


यदि आपका डेटा सेट इस नोटबुक से भिन्न है, तो आप केवल मामूली समायोजन के साथ इस नोटबुक का सीधे उपयोग कर सकते हैं। बेशक, आपके हार्डवेयर डिवाइस (कंप्यूटर मेमोरी, ग्राफिक्स कार्ड आकार) के आधार पर, आपको आउट-ऑफ-मेमोरी त्रुटियों से बचने के लिए बैच आकार को उचित रूप से समायोजित करने की आवश्यकता है।
उन तीन मापदंडों को सेट करें, फिर बाकी नोटबुक सुचारू रूप से चलनी चाहिए:

In [ ]:
# स्क्वाड_v2 सही या गलत के बराबर है अर्थात क्रमशः SQUAD v1 या SQUAD v2 का उपयोग करना।
# यदि आप अन्य डेटा सेट का उपयोग कर रहे हैं, तो True का अर्थ है कि मॉडल "अनिवार्य" प्रश्नों का उत्तर दे सकता है, अर्थात, कुछ प्रश्न उत्तर नहीं देते हैं, जबकि False का अर्थ है कि सभी प्रश्नों का उत्तर दिया जाना चाहिए।
squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## डेटा सेट लोड करें

हम डेटा डाउनलोड करने और हमें आवश्यक मूल्यांकन संकेतक प्राप्त करने के लिए (बेंचमार्क बेंचमार्क की तुलना में) [🤗 डेटासेट] (https://github.com/huggingface/datasets) लाइब्रेरी का उपयोग करेंगे।

इन दोनों कार्यों को `load_dataset` और `load_metric` फ़ंक्शन का उपयोग करके आसानी से पूरा किया जा सकता है।

In [ ]:
from datasets import load_dataset, load_metric

उदाहरण के लिए, हम इस नोटबुक में [SQUAD डेटासेट](https://rajpurkar.github.io/SQuAD-explorer/) का उपयोग करेंगे। इसी प्रकार, यह नोटबुक सभी डेटासेट लाइब्रेरीज़ में उपलब्ध कराए गए सभी प्रश्न और उत्तर डेटा सेटों के लिए भी अनुकूलित है।

यदि आप अपने स्वयं के डेटासेट (json या csv प्रारूप) का उपयोग कर रहे हैं, तो कस्टम लोड करने का तरीका जानने के लिए कृपया [डेटासेट दस्तावेज़] (https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) देखें डेटा सेट का. आपको प्रत्येक कॉलम के लिए उपयोग किए गए नामों को समायोजित करने की आवश्यकता हो सकती है।

In [ ]:
# डेटा डाउनलोड करें (सुनिश्चित करें कि नेटवर्क है)
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

इसके अलावा, आप हमारे द्वारा प्रदान किए गए [लिंक] (https://gas.graviti.cn/dataset/datawhale/SQuAD) से डेटा को डाउनलोड और डीकंप्रेस भी कर सकते हैं, और डीकंप्रेस्ड 2 json फ़ाइलों को `docs/अध्याय 4' में कॉपी कर सकते हैं। -/डेटासेट/स्क्वाड निर्देशिका में एनएलपी कार्यों को हल करने के लिए ट्रांसफॉर्मर का उपयोग करें, और फिर इसे निम्नलिखित कोड के साथ लोड करें।

In [ ]:
import os

data_path = './dataset/squad/'
path = os.path.join(data_path, 'squad.py')
cache_dir = os.path.join(data_path, 'cache')
data_files = {"train": os.path.join(data_path, "train-v1.1.json"), "validation": os.path.join(data_path, "dev-v1.1.json")}
datasets = load_dataset(path, data_files=data_files, cache_dir=cache_dir)

यह `डेटासेट` ऑब्जेक्ट एक [`डेटासेटडिक्ट`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict) संरचना है, जो क्रमशः प्रशिक्षण, सत्यापन और परीक्षण इस निर्देश की एक कुंजी के अनुरूप है।

In [ ]:
# निम्नलिखित डेटासेट और उनके गुण देखें
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

चाहे वह प्रशिक्षण सेट हो, सत्यापन सेट हो या परीक्षण सेट हो, प्रत्येक प्रश्न और उत्तर डेटा नमूने में तीन कुंजी होंगी: "संदर्भ", "प्रश्न" और "उत्तर"।

हम किसी नमूने का चयन करने के लिए सबस्क्रिप्ट का उपयोग कर सकते हैं।

In [ ]:
datasets["train"][0]
#answer उत्तर का प्रतिनिधित्व करता है
# संदर्भ एक पाठ खंड का प्रतिनिधित्व करता है
# प्रश्न एक प्रश्न का प्रतिनिधित्व करता है

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

उत्तरों की व्याख्या पर ध्यान दें। पाठ खंड में उत्तर पाठ देने के अलावा, उत्तर उत्तर का स्थान भी देता है (वर्ण से शुरू करके गणना की गई, उपरोक्त उदाहरण 515वां स्थान है)।

यह समझने के लिए कि डेटा कैसा दिखता है, निम्न फ़ंक्शन प्रदर्शित करने के लिए डेटा सेट से यादृच्छिक रूप से कुछ उदाहरणों का चयन करेगा।

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"], num_examples=2)

## प्रशिक्षण डेटा को प्रीप्रोसेस करना

मॉडल में डेटा फीड करने से पहले, हमें डेटा को प्रीप्रोसेस करना होगा। प्रीप्रोसेसिंग टूल को 'टोकनाइज़र' कहा जाता है। `टोकनाइज़र` पहले इनपुट को टोकनाइज़ करता है, फिर टोकन को प्री-मॉडल में आवश्यक संबंधित टोकन आईडी में परिवर्तित करता है, और फिर इसे मॉडल द्वारा आवश्यक इनपुट प्रारूप में परिवर्तित करता है।

डेटा प्रीप्रोसेसिंग के उद्देश्य को प्राप्त करने के लिए, हम अपने टोकननाइज़र को इंस्टेंट करने के लिए `AutoTokenizer.from_pretrained` विधि का उपयोग करते हैं, जो सुनिश्चित करता है:

- हमें एक टोकननाइज़र मिलता है जो पूर्व-प्रशिक्षित मॉडल वन-टू-वन से मेल खाता है।
- निर्दिष्ट मॉडल चेकपॉइंट के अनुरूप टोकननाइज़र का उपयोग करते समय, हम मॉडल द्वारा आवश्यक शब्दावली लाइब्रेरी भी डाउनलोड करते हैं, सटीक होने के लिए, टोकन शब्दावली।

डाउनलोड किए गए टोकन शब्दावली को कैश किया जाएगा ताकि दोबारा उपयोग करने पर इसे दोबारा डाउनलोड न किया जा सके।

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

निम्नलिखित कोड के लिए आवश्यक है कि टोकननाइज़र ट्रांसफार्मर.PreTrainedTokenizerFast प्रकार का होना चाहिए, क्योंकि हमें प्रीप्रोसेसिंग के दौरान फास्ट टोकननाइज़र की कुछ विशेष सुविधाओं (जैसे मल्टी-थ्रेडेड फास्ट टोकननाइज़र) का उपयोग करने की आवश्यकता होती है।

मॉडलों के अनुरूप लगभग सभी टोकननाइजरों में संगत तेज टोकननाइजर होते हैं। हम [मॉडल टोकनाइज़र कॉरेस्पोंडेंस टेबल] (https://huggingface.co/transformers/index.html#bigtable) में सभी पूर्व-प्रशिक्षित मॉडलों के अनुरूप टोकननाइज़र की विशेषताओं को देख सकते हैं।

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
# यदि हम टोकननाइज़र प्रीप्रोसेसिंग के बाद टेक्स्ट प्रारूप देखना चाहते हैं, तो हम केवल टोकननाइज़र की टोकननाइज़ विधि का उपयोग करते हैं। विशेष टोकन जोड़ने का अर्थ है पूर्व-प्रशिक्षित मॉडल के लिए आवश्यक विशेष टोकन जोड़ना।
print("单个文本tokenize: {}".format(tokenizer.tokenize("What is your name?"), add_special_tokens=True))
print("2个文本tokenize: {}".format(tokenizer.tokenize("My name is Sylvain.", add_special_tokens=True)))
# प्री-ट्रेनिंग मॉडल के लिए आवश्यक इनपुट प्रारूप टोकन आईडी है, और एक एटेटनियन मास्क भी आवश्यक है। पूर्व-प्रशिक्षित मॉडल प्रारूप द्वारा आवश्यक इनपुट प्राप्त करने के लिए आप निम्न विधि का उपयोग कर सकते हैं।

单个文本tokenize: ['what', 'is', 'your', 'name', '?']
2个文本tokenize: ['[CLS]', 'my', 'name', 'is', 'sy', '##lva', '##in', '.', '[SEP]']


टोकननाइज़र या तो एक टेक्स्ट या टेक्स्ट की एक जोड़ी को प्रीप्रोसेस कर सकता है। टोकननाइज़र प्रीप्रोसेसिंग के बाद प्राप्त डेटा प्री-ट्रेनिंग मॉडल के इनपुट प्रारूप से मिलता है।

In [ ]:
# एकल पाठ को प्रीप्रोसेस करें
tokenizer("What is your name?")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# दो टेक्स्ट को प्रीप्रोसेस करें। आप देख सकते हैं कि टोकनाइज़र शुरुआत में 101 टोकन आईडी जोड़ता है, टेक्स्ट के दो टुकड़ों को अलग करने के लिए बीच में 102 टोकन आईडी का उपयोग करता है, और अंत में 102 के साथ समाप्त होता है। ये नियम पूर्व-प्रशिक्षित मॉडलों के लिए डिज़ाइन किए गए हैं।
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

ऊपर देखी गई टोकन आईडी, जो कि इनपुट_आईडी हैं, आम तौर पर पूर्व-प्रशिक्षित मॉडल के नाम के साथ बदलती रहती हैं। इसका कारण यह है कि विभिन्न पूर्व-प्रशिक्षण मॉडल पूर्व-प्रशिक्षण के दौरान अलग-अलग नियम निर्धारित करते हैं। लेकिन जब तक टोकननाइजर और मॉडल के नाम सुसंगत हैं, तब तक टोकननाइजर द्वारा पूर्व-संसाधित इनपुट प्रारूप मॉडल की जरूरतों को पूरा करेगा। प्रीप्रोसेसिंग के बारे में अधिक जानकारी के लिए, कृपया [यह ट्यूटोरियल](https://huggingface.co/transformers/preprocessing.html) देखें

अब हमें यह भी सोचने की ज़रूरत है कि पूर्व-प्रशिक्षित मशीन प्रश्न उत्तर देने वाले मॉडल बहुत लंबे पाठ को कैसे संभालते हैं। सामान्यतया, पूर्व-प्रशिक्षित मॉडल इनपुट के लिए अधिकतम लंबाई की आवश्यकता होती है, इसलिए हम आमतौर पर अत्यधिक लंबे इनपुट को छोटा कर देते हैं। हालाँकि, यदि हम प्रश्न और उत्तर डेटा ट्रिपलेट <प्रश्न, संदर्भ, उत्तर> में बहुत लंबे संदर्भ को छोटा करते हैं, तो हम उत्तर खो सकते हैं (क्योंकि हम उत्तर के रूप में संदर्भ से एक छोटा सा टुकड़ा निकालते हैं)। इस समस्या को हल करने के लिए, नीचे दिया गया कोड एक उदाहरण ढूंढता है जो लंबाई से अधिक है और फिर आपको दिखाता है कि इसे कैसे संभालना है। हम सुपर लंबे इनपुट को कई छोटे इनपुट में काटते हैं, और प्रत्येक इनपुट को मॉडल की अधिकतम लंबाई इनपुट आवश्यकता को पूरा करना होगा। चूँकि उत्तर कहीं न कहीं स्लाइस के साथ मौजूद हो सकता है, हमें आसन्न स्लाइस के बीच प्रतिच्छेदन की अनुमति देने की आवश्यकता है, जो कोड में `doc_stride` पैरामीटर द्वारा नियंत्रित होता है।.

मशीन प्रश्न उत्तर देने वाले पूर्व-प्रशिक्षण मॉडल आमतौर पर प्रश्न और संदर्भ को इनपुट के रूप में जोड़ते हैं, और फिर मॉडल को संदर्भ से उत्तर ढूंढने देते हैं।

In [ ]:
max_length = 384 # 输入feature的最大长度，question和context拼接之后
doc_stride = 128 # 2个切片之间的重合token数量。

फॉर लूप एक अत्यंत लंबे नमूने को खोजने के लिए डेटा सेट को पार करता है। इस नोटबुक उदाहरण में मॉडल द्वारा आवश्यक अधिकतम इनपुट 384 है (512 भी आमतौर पर उपयोग किया जाता है)

In [ ]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

यदि छोटा नहीं किया गया है, तो इनपुट लंबाई 396 है

In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])

396

अब यदि हम 384 की अधिकतम लंबाई तक काट-छांट करते हैं, तो हम लंबे भाग की जानकारी खो देंगे

In [ ]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

ध्यान दें कि आम तौर पर बोलते हुए, हम केवल संदर्भ को काटते हैं, प्रश्न को नहीं क्योंकि संदर्भ प्रश्न के बाद जोड़ा जाता है और दूसरे पाठ से मेल खाता है, .tokenizer को नियंत्रित करने के लिए `only_third` का उपयोग किया जाता है और नियंत्रित करने के लिए `doc_stride` का उपयोग किया जाता है। टुकड़ा करना। उनके बीच ओवरलैप लंबाई।

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

बहुत लंबे इनपुट को स्लाइस करने के कारण, हमें कई इनपुट मिलते हैं, इन इनपुट इनपुट_आईडी की संबंधित लंबाई हैं

In [ ]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 157]

हम पूर्व-संसाधित टोकन आईडी और इनपुट_आईडी को टेक्स्ट प्रारूप में पुनर्स्थापित कर सकते हैं:

In [ ]:
for i, x in enumerate(tokenized_example["input_ids"][:2]):
    print("切片: {}".format(i))
    print(tokenizer.decode(x))

切片: 0
[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth a

चूँकि हमने बहुत लंबे पाठ को काट दिया है, इसलिए हमें उत्तर का स्थान (संदर्भ के प्रत्येक भाग की शुरुआत के सापेक्ष) फिर से खोजने की आवश्यकता है। मशीन प्रश्न उत्तर मॉडल प्रशिक्षण लेबल के रूप में उत्तर की स्थिति (उत्तर की प्रारंभिक स्थिति और अंतिम स्थिति, प्रारंभ और अंत) का उपयोग करेगा (उत्तर के टोकन आईडीएस के बजाय)। इसलिए, स्लाइसिंग के लिए मूल इनपुट के साथ एक संगत संबंध होना आवश्यक है, और स्लाइसिंग के बाद संदर्भ में प्रत्येक टोकन की स्थिति और मूल अल्ट्रा-लॉन्ग संदर्भ में स्थिति के बीच संबंधित संबंध होना चाहिए। आप इस पत्राचार का मानचित्र प्राप्त करने के लिए टोकननाइज़र में `रिटर्न_ऑफ़सेट्स_मैपिंग` पैरामीटर का उपयोग कर सकते हैं:

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
#स्लाइस से पहले और बाद में स्थिति सबस्क्रिप्ट के बीच संबंधित संबंध प्रिंट करें
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 8), (9, 13), (14, 18), (19, 22), (23, 28), (29, 33), (34, 37), (37, 38), (38, 39), (40, 50), (51, 55), (56, 60), (60, 61), (0, 0), (0, 3), (4, 7), (7, 8), (8, 9), (10, 20), (21, 25), (26, 29), (30, 34), (35, 36), (36, 37), (37, 40), (41, 45), (45, 46), (47, 50), (51, 53), (54, 58), (59, 61), (62, 69), (70, 73), (74, 78), (79, 86), (87, 91), (92, 96), (96, 97), (98, 101), (102, 106), (107, 115), (116, 118), (119, 121), (122, 126), (127, 138), (138, 139), (140, 146), (147, 153), (154, 160), (161, 165), (166, 171), (172, 175), (176, 182), (183, 186), (187, 191), (192, 198), (199, 205), (206, 208), (209, 210), (211, 217), (218, 222), (223, 225), (226, 229), (230, 240), (241, 245), (246, 248), (248, 249), (250, 258), (259, 262), (263, 267), (268, 271), (272, 277), (278, 281), (282, 285), (286, 290), (291, 301), (301, 302), (303, 307), (308, 312), (313, 318), (319, 321), (322, 325), (326, 330), (330, 331), (332, 340), (341, 351), (352, 354), (355, 363), (364, 373), (374,

ऊपर जो छपा है वह मूल संदर्भ स्लाइस मेंटोकनाइज्ड_उदाहरण के स्लाइस 0 के पहले 100 टोकन की स्थिति है। ध्यान दें कि पहला टोकन `[CLS]` (0, 0) पर सेट है क्योंकि यह टोकन प्रश्न या उत्तर का हिस्सा नहीं है। दूसरे टोकन के अनुरूप प्रारंभिक और अंतिम स्थिति 0 और 3 हैं। हम स्लाइसिंग के बाद टोकन आईडी के अनुसार संबंधित टोकन को परिवर्तित कर सकते हैं; फिर स्लाइसिंग से पहले टोकन स्थिति पर वापस मैप करने के लिए `ऑफसेट_मैपिंग` पैरामीटर का उपयोग करें और टोकन को मूल स्थिति में ढूंढें। चूँकि प्रश्न को संदर्भ के सामने रखा गया है, आप इसे सीधे सबस्क्रिप्ट के आधार पर प्रश्न से पा सकते हैं।

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

how How


इसलिए, हमें स्लाइसिंग से पहले और बाद में स्थिति का पत्राचार मिलता है। हमें प्रश्न और संदर्भ में अंतर करने के लिए `sequence_ids` पैरामीटर का भी उपयोग करने की आवश्यकता है।

In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

'कोई नहीं' विशेष टोकन से मेल खाता है, और फिर 0 या 1 अंक पहले पाठ और दूसरे पाठ का प्रतिनिधित्व करते हैं क्योंकि हम पहले क्यूस्टिन और दूसरे संदर्भ में पास होते हैं, वे क्रमशः प्रश्न और संदर्भ से मेल खाते हैं। अंत में, हम प्रीप्रोसेसिंग के बाद फीचर्स में एनोटेटेड उत्तर का स्थान पा सकते हैं:

In [ ]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# वर्तमान टेक्स्ट का स्टार्ट टोकन इंडेक्स ढूंढें।
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# वर्तमान पाठ का अंतिम टोकन आईडीनेक्स ढूंढें।
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# जांचें कि क्या उत्तर पाठ अंतराल के बाहर है, इस मामले में, इसका मतलब है कि नमूने का डेटा सीएलएस टोकन स्थिति पर चिह्नित है।
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
# टोकन_स्टार्ट_इंडेक्स और टोकन_एंड_इंडेक्स को उत्तर स्थिति के दोनों ओर ले जाएं।
# नोट: उत्तर अंत में सीमा स्थिति में है।
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print("start_position: {}, end_position: {}".format(start_position, end_position))
else:
    print("The answer is not in this feature.")

start_position: 23, end_position: 26


हमें उत्तर के स्थान को सत्यापित करने की आवश्यकता है। सत्यापन विधि है: संबंधित टोकन आईडी प्राप्त करने के लिए उत्तर के स्थान सबस्क्रिप्ट का उपयोग करें, फिर इसे टेक्स्ट में बदलें, और फिर इसकी तुलना मूल उत्तर से करें।

In [ ]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

over 1, 600
over 1,600


कभी-कभी प्रश्न को संदर्भ से जोड़ा जाता है, और कभी-कभी संदर्भ को प्रश्न से जोड़ा जाता है। विभिन्न मॉडलों की अलग-अलग आवश्यकताएं होती हैं, इसलिए हमें निर्दिष्ट करने के लिए `पैडिंग_साइड` पैरामीटर का उपयोग करने की आवश्यकता होती है।

In [ ]:
pad_on_right = tokenizer.padding_side == "right" #context在右边

अब, सभी चरणों को एक साथ रखें। ऐसे मामले के लिए जहां संदर्भ में कोई उत्तर नहीं है, हम सीधे उत्तर की चिह्नित प्रारंभिक स्थिति और समाप्ति स्थिति को सीएलएस की सबस्क्रिप्ट पर रखते हैं। यदि `allow_impossible_answers` पैरामीटर `गलत` है, तो इन अनुत्तरित नमूनों को छोड़ दिया जाएगा। संक्षिप्तता के लिए, आइए पहले हैंडल को फेंक दें।

In [ ]:
def prepare_train_features(examples):
# उदाहरणों को न केवल ट्रंकेट (छंटना) और पैडिंग (पूर्ण) करना आवश्यक है, बल्कि सभी जानकारी को बनाए रखना भी आवश्यक है, इसलिए स्लाइसिंग विधि का उपयोग किया जाता है।
# प्रत्येक लंबे पाठ उदाहरण को कई इनपुट में विभाजित किया जाएगा, और दो आसन्न इनपुट के बीच एक प्रतिच्छेदन होगा।
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

# हम स्लाइस आईडी को मूल आईडी पर मैप करने के लिए ओवरफ़्लो_टू_सैंपल_मैपिंग पैरामीटर का उपयोग करते हैं।
# उदाहरण के लिए, यदि 2 उदाहरणों को 4 टुकड़ों में काटा जाता है, तो संबंधित टुकड़े [0, 0, 1, 1] होते हैं, और पहले दो टुकड़े मूल पहले उदाहरण के अनुरूप होते हैं।
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
#ऑफ़सेट_मैपिंग भी 4 स्लाइस से मेल खाती है
#ऑफ़सेट_मैपिंग पैरामीटर हमें मूल इनपुट पर मैप करने में मदद करता है क्योंकि उत्तर मूल इनपुट पर एनोटेट किया गया है, यह हमें उत्तर की शुरुआती और अंतिम स्थिति ढूंढने में मदद करता है।
    offset_mapping = tokenized_examples.pop("offset_mapping")

# डेटा पुनः लेबल करें
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
# प्रत्येक स्लाइस को प्रोसेस करें
# सीएलएस के उत्तर के बिना नमूनों को चिह्नित करें
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

# प्रश्न और संदर्भ के बीच अंतर करें
        sequence_ids = tokenized_examples.sequence_ids(i)

# मूल उदाहरण सबस्क्रिप्ट प्राप्त करें.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
# यदि कोई उत्तर नहीं है, तो उत्तर के रूप में सीएलएस के स्थान का उपयोग करें।
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
# उत्तर की वर्ण स्तर प्रारंभ/अंत स्थिति।
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

# टोकन लेवल इंडेक्स प्रारंभ ढूंढें।
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

# टोकन लेवल इंडेक्स का अंत ढूंढें।
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

# जांचें कि क्या उत्तर पाठ की लंबाई से अधिक है। यदि यह अधिक है, तो सीएलएस सूचकांक का उपयोग एनोटेशन के रूप में भी किया जाएगा।
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
# यदि यह अधिक नहीं है, तो उत्तर टोकन की आरंभ और समाप्ति स्थिति ज्ञात करें। .
# नोट: यदि उत्तर अंतिम शब्द (एज केस) है तो हम अंतिम ऑफसेट के बाद जा सकते हैं।
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

उपरोक्त प्रीप्रोसेसिंग फ़ंक्शन एक नमूना या एकाधिक नमूना उदाहरणों को संसाधित कर सकता है। यदि कई नमूनों को संसाधित किया जाता है, तो कई नमूनों को पूर्व-संसाधित करने के बाद परिणामों की एक सूची वापस कर दी जाती है।

In [ ]:
features = prepare_train_features(datasets['train'][:5])
# 5 नमूनों की प्रक्रिया करें

इसके बाद, डेटासेट में सभी नमूनों को प्रीप्रोसेस करें। प्रसंस्करण विधि `मैप` फ़ंक्शन का उपयोग करना है और सभी नमूनों को (मैप) करने के लिए प्रीप्रोसेसिंग फ़ंक्शन `prepare_train_features` लागू करना है।

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

इससे भी बेहतर, अगली बार संसाधित होने पर पुनर्गणना से बचने के लिए लौटाए गए परिणाम स्वचालित रूप से कैश किए जाएंगे (लेकिन कृपया यह भी ध्यान दें कि यदि इनपुट बदल दिया जाता है, तो यह कैश से प्रभावित हो सकता है!)। डेटासेट लाइब्रेरी फ़ंक्शन यह निर्धारित करने के लिए इनपुट पैरामीटर का पता लगाएगा कि क्या परिवर्तन हैं। यदि कोई परिवर्तन नहीं हैं, तो कैश्ड डेटा का उपयोग किया जाएगा। यदि परिवर्तन हैं, तो डेटा को पुन: संसाधित किया जाएगा। लेकिन यदि इनपुट पैरामीटर अपरिवर्तित रहते हैं और आप इनपुट बदलना चाहते हैं, तो कैश को साफ़ करना और समायोजित करना सबसे अच्छा है। साफ़ करने का तरीका `load_from_cache_file=False` पैरामीटर का उपयोग करना है। इसके अलावा, ऊपर इस्तेमाल किया गया `बैच = ट्रू` पैरामीटर टोकननाइज़र की एक विशेषता है, जिसके बारे में माना जाता है कि यह एक ही समय में समानांतर में इनपुट को संसाधित करने के लिए कई थ्रेड्स का उपयोग करता है।

## फाइन-ट्यूनिंग फाइन-ट्यूनिंग मॉडल

वर्तमान में, हमने प्रशिक्षण/फाइन-ट्यूनिंग के लिए आवश्यक डेटा को पूर्व-प्रसंस्कृत किया है, और अब हम पूर्व-प्रशिक्षित मॉडल डाउनलोड करते हैं। चूँकि हम जो कर रहे हैं वह एक मशीनी प्रश्न उत्तर देने वाला कार्य है, इसलिए हम इस वर्ग `AutoModelForQuestionAnswering` का उपयोग करते हैं। टोकननाइज़र के समान, मॉडल को भी `from_pretrained` विधि का उपयोग करके लोड किया जाता है।

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading: 100%|██████████| 268M/268M [00:46<00:00, 5.79MB/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs

चूंकि हमारा फाइन-ट्यूनिंग कार्य एक मशीन प्रश्न उत्तर देने वाला कार्य है, और हम एक पूर्व-प्रशिक्षित भाषा मॉडल लोड कर रहे हैं, उपरोक्त हमें संकेत देगा कि मॉडल लोड करते समय कुछ बेमेल तंत्रिका नेटवर्क पैरामीटर हटा दिए गए थे (पूर्व का तंत्रिका नेटवर्क प्रमुख) -प्रशिक्षित भाषा मॉडल इसे फेंक दें, और मशीन प्रश्न उत्तर के तंत्रिका नेटवर्क प्रमुख को बेतरतीब ढंग से प्रारंभ करें)।

इन बेतरतीब ढंग से आरंभ किए गए मापदंडों के कारण, हमें अपने मॉडल को नए डेटा सेट पर फिर से ठीक करने की आवश्यकता है।

`ट्रेनर` प्रशिक्षण उपकरण प्राप्त करने के लिए, हमें 3 तत्वों की भी आवश्यकता है, जिनमें से सबसे महत्वपूर्ण है प्रशिक्षण सेटिंग्स/पैरामीटर [`ट्रेनिंगआर्ग्यूमेंट्स`](https://huggingface.co/transformers/main_classes/trainer.html# ट्रांसफार्मर। प्रशिक्षण तर्क)। इस प्रशिक्षण सेटिंग में वे सभी गुण शामिल हैं जो प्रशिक्षण प्रक्रिया को परिभाषित करते हैं। इसके लिए एक फ़ोल्डर नाम की भी आवश्यकता होती है. इस फ़ोल्डर का उपयोग मॉडल और अन्य मॉडल कॉन्फ़िगरेशन को सहेजने के लिए किया जाएगा।

In [ ]:
args = TrainingArguments(
    f"test-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5, #学习率
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3, # 训练的论次
    weight_decay=0.01,
)

उपरोक्त `मूल्यांकन_रणनीति = "युग" पैरामीटर प्रशिक्षण कोड बताता है: हम प्रत्येक युग के लिए एक सत्यापन मूल्यांकन करेंगे।

उपरोक्त `बैच_आकार` को इस नोटबुक से पहले परिभाषित किया गया था।

हम मॉडल में प्रीप्रोसेस्ड डेटा फीड करने के लिए default_data_colllator का उपयोग करते हैं।

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

ट्रेनिंग के दौरान हम सिर्फ नुकसान का आकलन करेंगे. मूल्यांकन मेट्रिक्स के आधार पर मॉडल का मूल्यांकन अगले भाग में छोड़ दिया जाएगा।

आपको केवल मॉडल, प्रशिक्षण पैरामीटर, डेटा, पहले उपयोग किए गए टोकननाइज़र और डेटा डिलीवरी टूल default_data_collater को ट्रानियर में पास करने की आवश्यकता है।

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

प्रशिक्षण शुरू करने के लिए `ट्रेन` विधि को कॉल करें

In [ ]:
trainer.train()

चूँकि प्रशिक्षण का समय बहुत लंबा है, यदि इसे स्थानीय मैक पर प्रशिक्षित किया जाता है, तो प्रत्येक ईपीको को गायब होने में लगभग 2 लगते हैं, इसलिए प्रत्येक प्रशिक्षण के बाद निम्नलिखित मॉडल सहेजा जाता है।

In [ ]:
trainer.save_model("test-squad-trained")

## मूल्यांकनमूल्यांकन

मॉडल मूल्यांकन थोड़ा अधिक जटिल होगा। हमें मॉडल के आउटपुट को उस टेक्स्ट प्रारूप में पोस्ट-प्रोसेस करना होगा जिसकी हमें आवश्यकता है। मॉडल स्वयं उत्तर की आरंभ/अंत स्थिति के लॉग की भविष्यवाणी करता है। यदि हम मूल्यांकन करते समय मॉडल को एक बैच खिलाते हैं, तो आउटपुट इस प्रकार होगा:

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

मॉडल का आउटपुट एक तानाशाही-जैसी डेटा संरचना है, जिसमें हानि (क्योंकि लेबल प्रदान किया गया है, सभी में हानि है), उत्तर प्रारंभ और अंत लॉग शामिल हैं। भविष्यवाणी के नतीजे निकालते समय हमें नुकसान को देखने की ज़रूरत नहीं है, बस सीधे लॉगिट्स को देखें।

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

प्रत्येक सुविधा में प्रत्येक टोकन में एक लॉगिट होगा। उत्तर की भविष्यवाणी करने का सबसे सरल तरीका उत्तर की वास्तविक स्थिति के रूप में प्रारंभ लॉग में सबसे बड़ी सबस्क्रिप्ट का चयन करना है, और उत्तर की अंतिम स्थिति के रूप में अंतिम लॉग में सबसे बड़ी सबस्क्रिप्ट का चयन करना है।

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 46,  57,  78,  43, 118,  15,  72,  35,  15,  34,  73,  41,  80,  91,
         156,  35], device='cuda:0'),
 tensor([ 47,  58,  81,  55, 118, 110,  75,  37, 110,  36,  76,  53,  83,  94,
         158,  35], device='cuda:0'))

उपरोक्त रणनीतियाँ अधिकांश मामलों में अच्छी हैं। हालाँकि, यदि हमारा इनपुट हमें बताता है कि उत्तर नहीं मिल सकता है: उदाहरण के लिए, प्रारंभ की स्थिति अंत की स्थिति से बड़ी है, या प्रारंभ और अंत की स्थिति प्रश्न का बिंदु है।

इस समय, सरल तरीका यह है कि हमें अपने उत्तर के रूप में दूसरी सबसे अच्छी भविष्यवाणी को चुनना होगा। यदि यह संभव नहीं है, तो हम तीसरी सबसे अच्छी भविष्यवाणी को देखेंगे, इत्यादि।

चूँकि उपरोक्त विधि से व्यवहार्य उत्तर खोजना आसान नहीं है, इसलिए हमें अधिक उचित विधि के बारे में सोचने की आवश्यकता है। हम एक नया स्कोर प्राप्त करने के लिए प्रारंभ और अंत के लॉगिट जोड़ते हैं, और फिर प्रारंभ और अंत के सर्वोत्तम `n_best_size` जोड़े को देखते हैं। संबंधित उत्तर `n_best_size` आरंभ और अंत जोड़े से प्राप्त किए जाते हैं, और फिर उत्तरों की जाँच यह देखने के लिए की जाती है कि क्या वे मान्य हैं, अंत में, उन्हें उनके अंकों के अनुसार रेट किया जाता है, और उच्चतम स्कोर वाले को उत्तर के रूप में चुना जाता है .

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# सर्वोत्तम आरंभ और अंत लॉग स्थान एकत्र करें:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # 如果start小雨end，那么合理的
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # 后续需要根据token的下标将答案找出来
                }
            )

फिर हम सर्वोत्तम उत्तर खोजने के लिए `valid_answers` को `स्कोर` के अनुसार क्रमबद्ध करते हैं। अंतिम चरण यह जांचना है कि क्या प्रारंभ और अंत स्थिति के अनुरूप पाठ संदर्भ में है और प्रश्न में नहीं है।

इसे पूरा करने के लिए, हमें सत्यापन सुविधाओं में निम्नलिखित दो जानकारी जोड़ने की आवश्यकता है:
- उदाहरण की आईडी जिसने सुविधा उत्पन्न की। चूँकि प्रत्येक उदाहरण कई सुविधाएँ उत्पन्न कर सकता है, प्रत्येक सुविधा/स्लाइस की विशेषताओं को उनके संबंधित उदाहरण जानने की आवश्यकता होती है।
- ऑफसेट मैपिंग: चरित्र की सबस्क्रिप्ट स्थिति के आधार पर प्रत्येक स्लाइस के टोकन की स्थिति को मूल पाठ में मैप करें।

इसलिए हमने निम्नलिखित सत्यापन सेट को पुन: संसाधित किया। प्रशिक्षण से निपटने के दौरान यह `prepare_train_features` से थोड़ा अलग है।

In [ ]:

def prepare_validation_features(examples):
# हमारे उदाहरणों को ट्रंकेशन और शायद पैडिंग के साथ टोकनाइज़ करें, लेकिन एक स्ट्राइड का उपयोग करके ओवरफ़्लो बनाए रखें
# एक उदाहरण में एक संदर्भ लंबा होने पर कई विशेषताएं देना संभव है, उनमें से प्रत्येक विशेषता में एक है
# संदर्भ जो पिछली सुविधा के संदर्भ को थोड़ा ओवरलैप करता है।
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

# चूंकि एक उदाहरण हमें कई विशेषताएं दे सकता है यदि इसका संदर्भ लंबा है, तो हमें एक सुविधा से लेकर एक मानचित्र की आवश्यकता है
# इसका संगत उदाहरण यह कुंजी हमें बस यही देती है।
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

# हम example_id रखते हैं जिसने हमें यह सुविधा दी है और हम ऑफसेट मैपिंग संग्रहीत करेंगे।
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
# उस उदाहरण के अनुरूप अनुक्रम को पकड़ें (यह जानने के लिए कि संदर्भ क्या है और प्रश्न क्या है)।
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

# एक उदाहरण कई स्पैन दे सकता है, यह टेक्स्ट के इस स्पैन वाले उदाहरण का सूचकांक है।
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

# ऑफसेट_मैपिंग को कोई नहीं पर सेट करें जो संदर्भ का हिस्सा नहीं है, इसलिए यह निर्धारित करना आसान है कि कोई टोकन है या नहीं
# स्थिति संदर्भ का हिस्सा है या नहीं.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

पहले की तरह प्रत्येक सत्यापन सेट नमूने पर `prepare_validation_features` फ़ंक्शन लागू करें।

In [ ]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

सभी पूर्वानुमान परिणाम प्राप्त करने के लिए `ट्रेनर.प्रिडिक्ट` पद्धति का उपयोग करें

In [ ]:
raw_predictions = trainer.predict(validation_features)

यह `ट्रेनर` *कुछ विशेषताओं को छुपाता है* जिनका उपयोग मॉडल प्रशिक्षण के दौरान नहीं किया जाता है (यहां `example_id` और `offset_mapping` हैं, जिनका उपयोग पोस्ट-प्रोसेसिंग के दौरान किया जाएगा), इसलिए हमें इन्हें वापस सेट करने की आवश्यकता है:

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

जब टोकन स्थिति प्रश्न भाग से मेल खाती है, तो `prepare_validation_features` फ़ंक्शन ऑफसेट मैपिंग को `कोई नहीं` पर सेट करता है, इसलिए हम ऑफसेट मैपिंग के आधार पर आसानी से पहचान सकते हैं कि टोकन संदर्भ में है या नहीं। हम विशेष रूप से लंबे उत्तरों को फेंकना भी समाप्त कर देते हैं।

In [ ]:
max_answer_length = 30

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# पहली विशेषता पहले उदाहरण से आती है। अधिक सामान्य मामले के लिए, हमें example_id का मिलान करना होगा
# एक उदाहरण सूचकांक
context = datasets["validation"][0]["context"]

# सर्वोत्तम प्रारंभ/अंत लॉग वाले सूचकांक एकत्रित करें:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
# दायरे से बाहर उत्तरों पर विचार न करें, क्योंकि या तो सूचकांक सीमा से बाहर हैं या अनुरूप हैं
# इनपुट_आईडी के उस भाग के लिए जो संदर्भ में नहीं है।
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
# ऐसे उत्तरों पर विचार न करें जिनकी लंबाई या तो < 0 या > max_answer_length हो।
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 16.706663, 'text': 'Denver Broncos'},
 {'score': 14.635585,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 13.234194, 'text': 'Carolina Panthers'},
 {'score': 12.468662, 'text': 'Broncos'},
 {'score': 11.709289, 'text': 'Denver'},
 {'score': 10.397583,
  'text': 'Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
 {'score': 10.104669,
  'text': 'American Football Conference (AFC) champion Denver Broncos'},
 {'score': 9.721636,
  'text': 'The American Football Conference (AFC) champion Denver Broncos'},
 {'score': 9.007437,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10'},
 {'score': 8.834958,
  'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina'},
 {'score': 8.38701,
  'text': 'Denver Broncos defeated the National Football Conference (NFC)'},
 {'score': 8.143825,
  'text': 'De

In [ ]:
将预测答案和真实答案进行比较即可:

In [ ]:
datasets["validation"][0]["answers"]

{'answer_start': [177, 177, 177],
 'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']}

आप देख सकते हैं कि मॉडल सही ढंग से बनाया गया है!

जैसा कि उपरोक्त उदाहरण में बताया गया है, चूँकि पहली विशेषता पहले उदाहरण से आनी चाहिए, यह अपेक्षाकृत आसान है। अन्य आशंकाओं के लिए, हमें विशेषताओं और उदाहरणों के मानचित्र की आवश्यकता है। इसी तरह, चूंकि एक उदाहरण को कई विशेषताओं में विभाजित किया जा सकता है, इसलिए हमें सभी सुविधाओं में सभी उत्तर एकत्र करने की भी आवश्यकता है। निम्नलिखित कोड एक्समैपल सबस्क्रिप्ट को फीचर्स सबस्क्रिप्ट में मैप करता है।

In [ ]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

पोस्ट-प्रोसेसिंग प्रक्रिया मूल रूप से पूरी हो चुकी है। आखिरी बात यह है कि उत्तर न देने की स्थिति को कैसे हल किया जाए (जब स्क्वाड_v2=सही हो)। उपरोक्त कोड केवल संदर्भ में उत्तरों पर विचार करता है, इसलिए हमें बिना उत्तर के पूर्वानुमान स्कोर भी एकत्र करने की आवश्यकता है (उत्तर के बिना पूर्वानुमान सीएलएसटी टोकन की शुरुआत और अंत के अनुरूप हैं)। यदि एक उदाहरण के नमूने में कई विशेषताएं हैं, तो हमें यह भी अनुमान लगाने की आवश्यकता है कि क्या कई विशेषताओं में कोई उत्तर नहीं है। इसलिए, बिना उत्तर का अंतिम स्कोर सभी सुविधाओं में सबसे कम बिना उत्तर वाला स्कोर होता है।

जब तक किसी भी उत्तर का अंतिम स्कोर अन्य सभी उत्तरों के स्कोर से अधिक नहीं होता, तब तक प्रश्न अनुत्तरित रहता है।

सब कुछ संयोजित करने के लिए:

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
# इसकी संबंधित विशेषताओं के लिए एक मानचित्र उदाहरण बनाएं।
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

# जो शब्दकोष हमें भरने हैं.
    predictions = collections.OrderedDict()

#लॉगिंग.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

# आइए सभी उदाहरणों पर नज़र डालें!
    for example_index, example in enumerate(tqdm(examples)):
# वे वर्तमान उदाहरण से जुड़ी सुविधाओं के सूचकांक हैं।
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
# वर्तमान उदाहरण से जुड़ी सभी सुविधाओं के माध्यम से लूपिंग।
        for feature_index in feature_indices:
# हम इस सुविधा के लिए मॉडल की भविष्यवाणियों को पकड़ते हैं।
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
# यही वह चीज़ है जो हमें मूल में पाठों की अवधि के लिए हमारे लॉग में कुछ स्थितियों को मैप करने की अनुमति देगी
# प्रसंग।
            offset_mapping = features[feature_index]["offset_mapping"]

# न्यूनतम शून्य पूर्वानुमान अपडेट करें.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

# `n_best_size` के बड़े प्रारंभ और अंत लॉग के लिए सभी संभावनाओं पर गौर करें।
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
# दायरे से बाहर उत्तरों पर विचार न करें, क्योंकि या तो सूचकांक सीमा से बाहर हैं या अनुरूप हैं
# इनपुट_आईडी के उस भाग के लिए जो संदर्भ में नहीं है।
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
# ऐसे उत्तरों पर विचार न करें जिनकी लंबाई या तो < 0 या > max_answer_length हो।
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
# अत्यंत दुर्लभ किनारे वाले मामले में हमारे पास एक भी गैर-शून्य भविष्यवाणी नहीं है, हम बचने के लिए एक नकली भविष्यवाणी बनाते हैं
# असफलता।
            best_answer = {"text": "", "score": 0.0}
        
# आइए अपना अंतिम उत्तर चुनें: सबसे अच्छा या शून्य उत्तर (केवल स्क्वाड_v2 के लिए)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

पोस्ट-प्रोसेसिंग फ़ंक्शन को मूल पूर्वानुमान पर लागू करें:

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 10570 example predictions split into 10784 features.


फिर हम मूल्यांकन मेट्रिक्स लोड करते हैं:

In [ ]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

उसी तरह, आप लोड करने के लिए हमारे द्वारा प्रदान की गई स्थानीय स्क्रिप्ट का भी उपयोग कर सकते हैं:

In [ ]:
metric_path = './dataset/metrics/squad.py'
metric = load_metric(metric_path)

फिर हम भविष्यवाणियों और टिप्पणियों के आधार पर मूल्यांकन मेट्रिक्स की गणना करते हैं। उचित तुलना के लिए, हमें पूर्वानुमानों और टिप्पणियों को प्रारूपित करने की आवश्यकता है। स्क्वाड2 के लिए, मूल्यांकन संकेतक को `no_answer_probability` पैरामीटर की भी आवश्यकता होती है (चूंकि कोई उत्तर नहीं है, यह सीधे एक खाली स्ट्रिंग पर सेट होता है, इसलिए यह पैरामीटर सीधे यहां 0.0 पर सेट होता है)

In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 76.74550614947965, 'f1': 85.13412652023338}

अंत में, [मॉडल अपलोड करने का तरीका देखें](https://huggingface.co/transformers/model_sharing.html) और मॉडल को [🤗 मॉडल हब](https://huggingface.co/models) पर अपलोड करना न भूलें। फिर आप अपने मॉडल का उपयोग सीधे नाम से कर सकते हैं, ठीक इस नोटबुक की शुरुआत की तरह।